# Requirements
- install python3.9
- install mysql server on Mac via homebrew `brew install mysql`
- start the mysql server `brew services start mysql`
- install the mysql python client `pip3 install pymysql`
- run this notebook (e.g.: VSCode's jupyter extension)

# Links
- installing python3.9: https://www.python.org/downloads/
- installing homebrew: https://brew.sh/
- installing VSCode and the jupyter extension: https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter

# Imports

In [ ]:
import pymysql
import pandas as pd
pd.options.mode.chained_assignment = None

# Open Connection

In [ ]:

connection = pymysql.connect(
	host='127.0.0.1',
	user='root',
	password= '',
	cursorclass=pymysql.cursors.DictCursor
)

cursor = connection.cursor()

#Create a database to run and store tables
cursor.execute("CREATE DATABASE IF NOT EXISTS mydatabase")
connection.select_db("mydatabase")
#execute a query to confirm first query worked
cursor.execute("SHOW DATABASES")
# print(cursor.fetchall()) #Verify that mydatabase appears in the list when print statement is run

# Extraction of Data from API's and CSV's

In [ ]:
#EXTRACTION

collision_data = pd.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv")
features = ["collision_id", "crash_time", "borough", "zip_code", "contributing_factor_vehicle_1", "contributing_factor_vehicle_2", "number_of_persons_injured", "number_of_persons_killed","crash_date"]
collision_data = collision_data.loc[:,features]
collision_data

In [ ]:
uber_data = pd.read_csv("uber_nyc_enriched.csv")  
features = ["pickups", "spd", "vsb", "temp", "pcp01", "pcp06", "pcp24", "sd", "borough", "hday", "pickup_dt"]
uber_data = uber_data.loc[:,features]

uber_data.insert(0, "uber_id", [i for i in range(len(uber_data))])
uber_data

In [ ]:
noise_incident_data = pd.read_csv("party_in_nyc.csv")  
features = ["Created Date", "Incident Zip", "Borough", "Location Type"]
noise_incident_data = noise_incident_data.loc[:,features]

noise_incident_data.insert(0, "incident_id", [i for i in range(len(noise_incident_data))])
noise_incident_data

# Transformation of Data

In [ ]:
#remove NaN values
collision_data.dropna(inplace=True)

#Concatenate crash_time and crash_date into one column, convert to proper SQL date-time format, and drop the crash_time column
collision_data['crash_date'] = collision_data['crash_date'].str[:10]
collision_data['crash_date'] = collision_data['crash_date'] + ' ' +  collision_data['crash_time']
collision_data['crash_date'] = pd.to_datetime(collision_data['crash_date'], format='%Y-%m-%d %H:%M')
del collision_data['crash_time']

#change data types
collision_data.dtypes
collision_data['collision_id'] = collision_data['collision_id'].astype(int)
collision_data['borough'] = collision_data['borough'].astype(str)
collision_data['zip_code'] = collision_data['zip_code'].astype(int)
collision_data['contributing_factor_vehicle_1'] = collision_data['contributing_factor_vehicle_1'].astype(str)
collision_data['contributing_factor_vehicle_2'] = collision_data['contributing_factor_vehicle_2'].astype(str)
collision_data['number_of_persons_injured'] = collision_data['number_of_persons_injured'].astype(int)
collision_data['number_of_persons_killed'] = collision_data['number_of_persons_killed'].astype(int)

collision_data

In [ ]:
collision_data.columns

In [ ]:
#remove unnecessary value in borough
noise_incident_data.Borough.unique()
noise_incident_data = noise_incident_data[noise_incident_data.Borough != 'Unspecified']

#change data types
noise_incident_data.dtypes
noise_incident_data['incident_id'] = noise_incident_data['incident_id'].astype(int)
noise_incident_data['Incident Zip'] = noise_incident_data['Incident Zip'].astype(float)
noise_incident_data['Borough'] = noise_incident_data['Borough'].astype(str)
noise_incident_data['Location Type'] = noise_incident_data['Location Type'].astype(str)
noise_incident_data['Created Date'] = pd.to_datetime(noise_incident_data['Created Date'], format = '%Y-%m-%d %H:%M:%S')





In [ ]:
noise_incident_data

In [ ]:
#Convert to date time
uber_data['pickup_dt'] = pd.to_datetime(uber_data['pickup_dt'], format = '%m/%d/%y %H:%M') #1/1/15 1:00

#remove NaN values
uber_data.dropna(inplace=True)

#remove unnecessary boroughs
uber_data.borough.unique()
uber_data = uber_data[uber_data.borough != 'EWR']

#match borough text with other dataframes
uber_data['borough'] = uber_data['borough'].str.upper()

#change data types
uber_data.dtypes
uber_data['uber_id'] = uber_data['uber_id'].astype(int)
uber_data['pickups'] = uber_data['pickups'].astype(int)
uber_data['borough'] = uber_data['borough'].astype(str)
uber_data['hday'] = uber_data['hday'].astype(str)
uber_data


In [ ]:
uber_data

## Creating the fact dataframe

### define the size of the dataframe to be the smallest size between all the referenced tables so that there are no missing values

In [ ]:
fact_table_size = min(
    len(collision_data),
    len(uber_data),
    len(noise_incident_data)
)

### create table

In [ ]:
facts = pd.DataFrame(data=uber_data['uber_id'])
facts.insert(1,'collision_id',collision_data['collision_id'])
facts.insert(2,'incident_id',noise_incident_data['incident_id'])
facts = facts.loc[:fact_table_size]
facts